# Neo4j data loading setup
Using neo4j, we’ll load in twitter data to create a network showing the relationships between legislators’ tweets, other users they mention, and hashtags they mention. 

***

`NOTEBOOK SETUP`
- Create a .py file containing the following declarations and place it in the same directory as this notebook
    - password = `Your neo4j server password`
    - NEO4J_DATA_DIR = `C:/Users/PC USERNAME/AppData/Local/Neo4j/Relate/Data/dbmss/SERVER ID/import/data/`
- Place the legislator dataset into the above NEO4J_DATA_DIR
- Set the ROOT_DIR directory to reflect where this notebook is in relation to the project root

***

In [1]:
import pandas as pd
import numpy as np
import sys
import os

In [2]:
ROOT_DIR = "../../"
DATA_DIR = ROOT_DIR + "data/"

In [3]:
# Load utils
sys.path.insert(0, os.path.abspath(ROOT_DIR + "/src"))
from utils import *

### Import local password and NEO4J_DATA_DIR

In [16]:
from pword import *

### Connect to local neo4j database

In [5]:
from neo4j import GraphDatabase
uri = 'neo4j://localhost:7687'
driver = GraphDatabase.driver(uri, auth=("neo4j", password))
session = driver.session()

***

# Data loading + processing

In [17]:
legislators_fname = DATA_DIR + "legislators-current.csv"
full_tweets_fname = DATA_DIR + "tweets.json"
tweets_fname = NEO4J_DATA_DIR + "legis_tweets.csv"

### Load legislators

In [7]:
legis_df = pd.read_csv(legislators_fname)

In [8]:
legis_df

,last_name,first_name,middle_name,suffix,nickname,full_name,birthday,gender,type,state,...,opensecrets_id,lis_id,fec_ids,cspan_id,govtrack_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id
0,Brown,Sherrod,NaN,NaN,NaN,Sherrod Brown,1952-11-09,M,sen,OH,...,N00003535,S307,"H2OH13033,S6OH00163",5051.0,400050,27018.0,Sherrod Brown,NaN,29389.0,Sherrod Brown
1,Cantwell,Maria,NaN,NaN,NaN,Maria Cantwell,1958-10-13,F,sen,WA,...,N00007836,S275,"S8WA00194,H2WA01054",26137.0,300018,27122.0,Maria Cantwell,NaN,39310.0,Maria Cantwell
2,Cardin,Benjamin,L.,NaN,NaN,Benjamin L. Cardin,1943-10-05,M,sen,MD,...,N00001955,S308,"H6MD03177,S6MD03177",4004.0,400064,26888.0,Ben Cardin,NaN,15408.0,Ben Cardin
3,Carper,Thomas,Richard,NaN,NaN,Thomas R. Carper,1947-01-23,M,sen,DE,...,N00012508,S277,S8DE00079,663.0,300019,22421.0,Tom Carper,NaN,15015.0,Tom Carper
4,Casey,Robert,P.,Jr.,Bob,"Robert P. Casey, Jr.",1960-04-13,M,sen,PA,...,N00027503,S309,S6PA00217,47036.0,412246,2541.0,"Bob Casey, Jr.",NaN,40703.0,Bob Casey Jr.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,Loeffler,Kelly,NaN,NaN,NaN,Kelly Loeffler,1970-11-27,F,sen,GA,...,NaN,S405,S0GA00526,NaN,456790,NaN,Kelly Loeffler,NaN,NaN,Kelly Loeffler
533,Mfume,Kweisi,NaN,NaN,NaN,Kweisi Mfume,1948-10-24,M,rep,MD,...,N00001799,NaN,"S6MD03185,H6MD07020,H0MD07114",NaN,407672,NaN,NaN,NaN,15433.0,Kweisi Mfume
534,Tiffany,Thomas,P.,NaN,NaN,NaN,1957-12-30,M,rep,WI,...,N00045307,NaN,H0WI07101,NaN,456791,NaN,Tom Tiffany,NaN,NaN,Tom Tiffany
535,Garcia,Mike,NaN,NaN,NaN,NaN,1976-04-24,M,rep,CA,...,N00044298,NaN,H0CA25105,NaN,456792,NaN,Mike Garcia,NaN,NaN,Mike Garcia (politician)


### Create tweets subset for only legislators present in the dataset

In [9]:
import datetime

In [10]:
def date_convert(dt):
    """Convert pandas datetime object to neo4j date string representation"""
    return f"{dt.year}-{dt.month}-{dt.day}"
#     if not isinstance(string, str):
#         return string
#     date = string.split(' ')[0]    
#     dt = datetime.datetime.strptime(date, "%m/%d/%Y")
#     return f"{dt.year}-{dt.month}-{dt.day}"

In [11]:
legis_tweets = []
legislator_names = set(legis_df.twitter.unique())
loaded_rows = 0

for chunk in pd.read_json(full_tweets_fname, lines=True, chunksize=1000):
    loaded_rows += chunk.shape[0]
    print(f"Processed {loaded_rows} rows", end='\r')
    for row in range(chunk.shape[0]):
        replier = chunk.iloc[row].screen_name
        repliee = chunk.iloc[row].in_reply_to_screen_name
        if replier in legislator_names and repliee in legislator_names:
            row = chunk.iloc[row].to_dict()
            # Remove all single and double quotes from text
            row["text"] = row["text"].replace('\'', '').replace("\"", "").replace('’', '').replace('\\', '/')
            # Add a hashtag key
            row["htags"] = row["entities"]["hashtags"]
            # Add a user_mentions key
            row["user_mentions"] = row["entities"]["user_mentions"]
            # Change created date to be default neo4j compatible
            row["created_at"] = date_convert(row["created_at"])
            
            legis_tweets.append(row)

In [13]:
legis_tweets_df = pd.DataFrame(legis_tweets)

In [14]:
legis_tweets_df.shape

(14988, 34)

In [18]:
legis_tweets_df.to_csv(tweets_fname, index=False)

***

# Neo4j data loading

### Load legislators into neo4j

In [19]:
q = """
WITH \"file:///""" + legislators_fname + """\" AS url
LOAD CSV WITH HEADERS FROM url AS row

CREATE (p:Person {fname:row.first_name, lname:row.last_name, full_name:row.full_name, twitter:row.twitter, party:row.party, state:row.state})

RETURN p
"""
out = session.run(q)
out.data()

[{'p': {'fname': 'Sherrod',
   'twitter': 'SenSherrodBrown',
   'lname': 'Brown',
   'full_name': 'Sherrod Brown',
   'state': 'OH',
   'party': 'Democrat'}},
 {'p': {'fname': 'Maria',
   'twitter': 'SenatorCantwell',
   'lname': 'Cantwell',
   'full_name': 'Maria Cantwell',
   'state': 'WA',
   'party': 'Democrat'}},
 {'p': {'fname': 'Benjamin',
   'twitter': 'SenatorCardin',
   'lname': 'Cardin',
   'full_name': 'Benjamin L. Cardin',
   'state': 'MD',
   'party': 'Democrat'}},
 {'p': {'fname': 'Thomas',
   'twitter': 'SenatorCarper',
   'lname': 'Carper',
   'full_name': 'Thomas R. Carper',
   'state': 'DE',
   'party': 'Democrat'}},
 {'p': {'fname': 'Robert',
   'twitter': 'SenBobCasey',
   'lname': 'Casey',
   'full_name': 'Robert P. Casey, Jr.',
   'state': 'PA',
   'party': 'Democrat'}},
 {'p': {'fname': 'Dianne',
   'twitter': 'SenFeinstein',
   'lname': 'Feinstein',
   'full_name': 'Dianne Feinstein',
   'state': 'CA',
   'party': 'Democrat'}},
 {'p': {'fname': 'Amy',
   'twitt

### Load tweets into neo4j

In [20]:
def upload_chunk(fname):
    q = """
    WITH \"file:///""" + fname + """\" AS url
    LOAD CSV WITH HEADERS FROM url AS row
    
    WITH row WHERE NOT row.in_reply_to_screen_name IS NULL
    
    MERGE (p1:Person {twitter:row.screen_name})
    MERGE (p2:Person {twitter:row.in_reply_to_screen_name})
        
    CREATE (p1) - [r:REPLIED] -> (p2)
    
    SET r.create_date = date(row.created_at)
    SET r.text = row.text
    
    RETURN r
    """
    out = session.run(q)
    
    return out.data()

In [21]:
chunk_num = 0
rows_uploaded = 0
# NEO4J_DATA_DIR is defined in pword.py, stored in the same directory as this notebook.
if not os.path.exists(NEO4J_DATA_DIR + "temp/"):
    os.mkdir(NEO4J_DATA_DIR + "temp/")

for chunk in pd.read_csv(tweets_fname, chunksize=10000):
    fname = "temp/" + f"tweets_chunk_{chunk_num}.csv"
    print(" "*50, end='\r')
    print("Saving temp file", end='\r')
    chunk.to_csv(NEO4J_DATA_DIR + fname, index=False)
    print(" "*50, end='\r')
    print(f"Uploading chunk {chunk_num} data", end='\r')
    upload_chunk(DATA_DIR + fname)
    print(" "*50, end='\r')
    print("Removing temp file", end='\r')
    os.remove(NEO4J_DATA_DIR + fname)
    
    chunk_num += 1
    rows_uploaded += chunk.shape[0]
    
print("Tweet upload complete")
print(f"{chunk_num + 1} chunks processed")
print(f"{rows_uploaded} rows uploaded")

Tweet upload complete                             
3 chunks processed
14988 rows uploaded


In [22]:
q = """
MATCH (p1:Person {twitter:'SenSherrodBrown'}) - [r:REPLIED] -> (p2:Person)
WHERE r.create_date > DATE("2012-1-1")
RETURN r.text
"""
out = session.run(q)

In [23]:
out = out.data()

In [24]:
len(out)

100

In [25]:
out[0]

{'r.text': '@SenKamalaHarris Were just getting warmed up - see you Sunday. #DefendTheLand'}